In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from vivarium_research_prl.noise import corruption, fake_names, noisify

!date
!whoami
!uname -a
!pwd

Sat 14 Jan 2023 11:11:26 AM PST
ndbs
Linux int-slurm-sarchive-p0008 5.4.0-88-generic #99-Ubuntu SMP Thu Sep 23 17:29:00 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/noise


In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
rng = np.random.default_rng(983292329775)

# Stuff

In [3]:
s = pd.Series(list('abdciaalgkaghha'))
s

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [5]:
sc = s.copy()
sc.loc[s.index]

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [15]:
a = rng.choice(list('abc'), len(s))
a

array(['a', 'b', 'b', 'a', 'b', 'a', 'a', 'b', 'b', 'a', 'b', 'a', 'b',
       'b', 'c'], dtype='<U1')

In [16]:
a == s

0      True
1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [11]:
type(a==s)

pandas.core.series.Series

In [21]:
u = (a==s)
s[u]

0    a
1    b
5    a
6    a
dtype: object

In [22]:
a[u]

array(['a', 'b', 'a', 'a'], dtype='<U1')

In [23]:
s[~u]

2     d
3     c
4     i
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [24]:
a[~u]

array(['b', 'a', 'b', 'b', 'b', 'a', 'b', 'a', 'b', 'b', 'c'], dtype='<U1')

# Test

In [132]:
rc = corruption.random_choice(s, list('ab'), random_state=54321)
rc

0     a
1     b
2     b
3     b
4     b
5     a
6     b
7     a
8     a
9     a
10    b
11    a
12    b
13    b
14    b
dtype: object

In [133]:
s == rc

0      True
1      True
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [134]:
rc2 = corruption.random_choice(s, list('ab'), exclude_current=True, random_state=54321)
rc2

0     b
1     a
2     b
3     b
4     b
5     b
6     b
7     a
8     a
9     a
10    b
11    a
12    b
13    b
14    b
dtype: object

In [135]:
s == rc2

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [47]:
s2 = pd.Series(rng.choice(['M', 'F'], 15))
s2

0     F
1     M
2     M
3     M
4     F
5     F
6     F
7     M
8     M
9     M
10    F
11    M
12    F
13    M
14    F
dtype: object

In [136]:
t2 = corruption.random_choice(s2, ['M', 'F'], random_state=85737)
t2

0     M
1     F
2     M
3     M
4     M
5     F
6     F
7     F
8     F
9     M
10    F
11    M
12    M
13    F
14    M
dtype: object

In [137]:
s2==t2

0     False
1     False
2      True
3      True
4     False
5      True
6      True
7     False
8     False
9      True
10     True
11     True
12    False
13    False
14    False
dtype: bool

In [56]:
(s2==t2).sum()

7

In [138]:
t2b = corruption.random_choice(s2, ['M', 'F'], exclude_current=True, random_state=85737)
t2b

0     M
1     F
2     F
3     F
4     M
5     M
6     M
7     F
8     F
9     F
10    M
11    F
12    M
13    F
14    M
dtype: object

In [139]:
s2==t2b

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

# Test again

In [140]:
size = int(1e6)
s3 = pd.Series(rng.choice(list('abcdef'), size))
s3

0         f
1         f
2         c
3         c
4         a
         ..
999995    a
999996    f
999997    e
999998    d
999999    a
Length: 1000000, dtype: object

In [141]:
t3a = corruption.random_choice(s3, s3.unique(), random_state=rng)
t3a

0         e
1         e
2         c
3         c
4         a
         ..
999995    a
999996    c
999997    f
999998    f
999999    e
Length: 1000000, dtype: object

In [142]:
same3a = (t3a == s3)
print(same3a.sum(), same3a.sum()/size)

167018 0.167018


In [143]:
t3b = corruption.random_choice(s3, s3.unique(), exclude_current=True, random_state=rng)
t3b

0         d
1         b
2         d
3         d
4         b
         ..
999995    c
999996    c
999997    f
999998    c
999999    d
Length: 1000000, dtype: object

In [144]:
same3b = (t3b == s3)
print(same3b.sum(), same3b.sum()/size)

0 0.0


# Test 2nd version

In [83]:
np.full(5, 7)

array([7, 7, 7, 7, 7])

In [85]:
val = 'b'
choices = list('abc')
p = [1/2, 1/3, 1/6]
choices, p = map(np.asarray, (choices, p))
np.where(choices != val, p, 0)

array([0.5       , 0.        , 0.16666667])

In [86]:
p_cond = np.where(choices != val, p, 0)
p_cond /= p_cond.sum()
p_cond

array([0.75, 0.  , 0.25])

In [145]:
for _ in range(10):
    print(corruption.random_choice2(6, [1,3,6]), sep=' ', end=' ')
print()
for _ in range(10):
    print(corruption.random_choice2(6, [1,3,6], exclude_current=True), sep=' ', end=' ')

1 3 1 1 3 3 6 1 6 3 
3 1 3 1 1 3 1 1 3 1 

In [146]:
sixes = pd.Series(6, index=range(10))
corruption.random_choice2(sixes, [1,3,6], random_state=8888)

0    1
1    1
2    3
3    1
4    3
5    6
6    3
7    6
8    6
9    3
dtype: int64

In [147]:
corruption.random_choice2(sixes, [1,3,6], exclude_current=True, random_state=8888)

0    1
1    1
2    3
3    3
4    1
5    1
6    1
7    1
8    3
9    1
dtype: int64

In [113]:
x = pd.Series(s, index=range(14,0,-1))
x

14    a
13    h
12    h
11    g
10    a
9     k
8     g
7     l
6     a
5     a
4     i
3     c
2     d
1     b
dtype: object

In [114]:
s

0     a
1     b
2     d
3     c
4     i
5     a
6     a
7     l
8     g
9     k
10    a
11    g
12    h
13    h
14    a
dtype: object

In [115]:
s.to_numpy()

array(['a', 'b', 'd', 'c', 'i', 'a', 'a', 'l', 'g', 'k', 'a', 'g', 'h',
       'h', 'a'], dtype=object)

In [120]:
i = pd.Series(rng.choice([True, False], len(t2)), index=t2)
i

M     True
F     True
M    False
M    False
M     True
F     True
F     True
F     True
F     True
M     True
F     True
M     True
M    False
F    False
M     True
dtype: bool

In [121]:
s.to_numpy()[i]

array(['a', 'b', 'i', 'a', 'a', 'l', 'g', 'k', 'a', 'g', 'a'],
      dtype=object)

In [ ]:
np.empty()

In [127]:
s.dtypes

dtype('O')

In [128]:
s.dtype

dtype('O')

# Test after refactoring

In [152]:
for _ in range(10):
    print(corruption.random_choice(3, [1,3,6]), sep=' ', end=' ')
print()
for _ in range(10):
    print(corruption.random_choice(3, [1,3,6], exclude_current=True), sep=' ', end=' ')

3 1 1 6 1 3 1 3 3 6 
1 6 1 1 1 6 6 6 1 1 